In [2]:
#Import libraries

import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

from keras import layers
from keras import models
from keras import optimizers
from keras import backend as K
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras.utils import Sequence
from keras.models import load_model
from tensorflow.distribute import MirroredStrategy

In [3]:
#All paths

Path = "/home/ug-ml/felix-ML/DataGenerator2/Data" #Path where training and validation data is
SaveDataPath = "/home/ug-ml/Documents/GitHub_BigFiles/SaveFolder" #Base directory of place you store information of models
SaveFolderName = "/Convnet5" #Will create a folder and put in information about the outcome / inputs
CifFlolder = "/home/ug-ml/felix-ML/DataGenerator2/CifFolder"
ModelName = "/Convnet5.hdf5"

batch_size = 32
LatticeRange = [6, 11]


In [4]:
#Load data generators

class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, file_type):
        """Here self.x is a list of paths to file_type files. self.y is a
        corresponding list of labels."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.file_type = file_type

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return arrs_from_paths(batch_x, self.file_type), to_categorical(np.array(batch_y),10)

def arrs_from_paths(paths, file_type):
    if file_type == "txt":
        return np.array([np.loadtxt(file_name) for file_name in paths])
    elif file_type == "npy":
        return np.array([np.load(file_name) for file_name in paths]) 
    
    
#Define Data gemerator
rng = np.random.default_rng()

PathOfFile = CifFlolder +"/FilePaths.txt"
with open(PathOfFile) as textFile:
    lines = [line.split() for line in textFile]


training_path = []
training_labels = []
validation_path = []
validation_labels = []
for i in lines:
    PathSplit = i[0].split("/")
    for j in PathSplit:
        if(j == "training"):
            if(float(i[3]) > LatticeRange[0] and float(i[3]) < LatticeRange[1]):
                training_path.append(i[0])
                training_labels.append(int(i[2]))
                break
            
        elif(j == "validation"):
            if(float(i[3]) > LatticeRange[0] and float(i[3]) < LatticeRange[1]):
                validation_path.append(i[0])
                validation_labels.append(int(i[2]))
                break

                
trainsize = len(training_path)
validationsize = len(validation_path)

ShuffleTraining = np.arange(trainsize, dtype = np.int)
ShuffleValidation = np.arange(validationsize, dtype = np.int)

training_path_shuffled = training_path.copy()
training_labels_shuffled = training_labels.copy()
validation_path_shuffled = validation_path.copy()
validation_labels_shuffled = validation_labels.copy()

rng.shuffle(ShuffleTraining)
rng.shuffle(ShuffleValidation)

for i in range(0, trainsize):
    training_path_shuffled[i] = training_path[ShuffleTraining[i]]
    training_labels_shuffled[i] = training_labels[ShuffleTraining[i]]
    #print(training_path_shuffled[i], training_labels_shuffled[i])

for i in range(0, validationsize):
    validation_path_shuffled[i] = validation_path[ShuffleValidation[i]]
    validation_labels_shuffled[i] = validation_labels[ShuffleValidation[i]]
    #print(validation_path_shuffled[i], validation_labels_shuffled[i])







print("data done.")
training_seq = FelixSequence(training_path_shuffled, training_labels_shuffled, batch_size, "npy")
print("training_seq done.")
val_images = arrs_from_paths(validation_path_shuffled, "npy")
print("val_images done.")
val_lab = to_categorical(validation_labels_shuffled)
print("val_lab done.")
print("Complete")

data done.
training_seq done.
val_images done.
val_lab done.
Complete


In [5]:
model = load_model(SaveDataPath + SaveFolderName + ModelName)
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_15 (Separab (None, 128, 126, 126)     5060      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 128, 63, 63)       0         
_________________________________________________________________
separable_conv2d_16 (Separab (None, 128, 61, 61)       17664     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 128, 30, 30)       0         
_________________________________________________________________
separable_conv2d_17 (Separab (None, 128, 28, 28)       17664     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 128, 14, 14)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)            

In [6]:
AllOutputs = np.zeros(validationsize * 10, dtype = np.float32).reshape(validationsize, 10)


Input = np.zeros(1 * 36 * 128 * 128, dtype=np.float32).reshape(1, 36, 128, 128)
Outputs = np.zeros(validationsize * 10, dtype=np.float32).reshape(validationsize, 10)

'''    
for i in range(0, validationsize):
    for layer in model.layers:
        Input[0] = val_images[i]
        keras_function = K.function([model.input], [layer.output])

        outputs.append(keras_function([Input, 1]))
    print(i)
print(outputs)

'''
print(validationsize)
for i in range(0, validationsize):
    for layer in model.layers:
        Input[0] = val_images[i]
        keras_function = K.function([model.input], [layer.output])

        A = keras_function([Input, 1])
        if(layer == model.layers[-1]):
            Outputs[i] = A[0][0]
            print(i)
print(Outputs)

1210
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [7]:
def ErrorEstimate(output):
    sum = 0
    maxval = 0
    maxvalarg = max(output)
    for i in range(0, 10):
        sum+= i * output[i]
        if(output[i] == maxvalarg):
            maxval = i

            
    Average = sum
    
    rms_sum = 0
    for i in range(0, 10):
        rms_sum+=((Average - i) * output[i]) ** 2
    rms = rms_sum ** 0.5
    #print(maxval, Average, rms)
    return(maxval, Average, rms)


In [9]:
n = 9
#print(Outputs[n])
sum_average = 0
sum_class = 0
for i in range(0, validationsize):
    A = ErrorEstimate(Outputs[i])
    sum_average+=(A[1] - validation_labels_shuffled[i]) ** 2
    sum_class+=(A[0] - validation_labels_shuffled[i]) ** 2
    print("Correct answer ", validation_labels_shuffled[i])
    print("Predicted ", A[0])
    print("Average ", A[1])
    print("Deviation ", A[2])
    print("Number of SD away", (A[1] - validation_labels_shuffled[i]) / A[2])
    print("")
    
print("Average", (sum_average / validationsize) ** 0.5)
print("Class", (sum_class / validationsize) ** 0.5)

Correct answer  6
Predicted  6
Average  5.956770535739124
Deviation  0.06476201826259889
Number of SD away -0.6675126165090791

Correct answer  4
Predicted  4
Average  4.00334473141283
Deviation  0.004591969176019438
Number of SD away 0.7283871656406639

Correct answer  9
Predicted  9
Average  8.99968861016809
Deviation  0.0004346166581590094
Number of SD away -0.7164700801614851

Correct answer  7
Predicted  8
Average  8.168523780525895
Deviation  0.2925376442261012
Number of SD away 3.994439018667791

Correct answer  5
Predicted  8
Average  7.4149268662918075
Deviation  0.65864458267154
Number of SD away 3.6665098747135816

Correct answer  4
Predicted  4
Average  4.015320991918915
Deviation  0.021351626290978802
Number of SD away 0.7175562043903055

Correct answer  1
Predicted  1
Average  1.0023707986199515
Deviation  0.0031963454158796894
Number of SD away 0.7417216575446374

Correct answer  0
Predicted  0
Average  0.000569454921237611
Deviation  0.0007944515753311698
Number of SD a

/home/ug-ml/felix-ML/env/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


 6
Predicted  6
Average  6.00099864443747
Deviation  0.0014094774155627553
Number of SD away 0.7085210635112876

Correct answer  3
Predicted  3
Average  3.0000028254045046
Deviation  3.6254985857463564e-06
Number of SD away 0.7793147446501979

Correct answer  8
Predicted  8
Average  8.00437743503672
Deviation  0.0064177574477371655
Number of SD away 0.68208171972327

Correct answer  5
Predicted  5
Average  5.000652006602035
Deviation  0.0009024166193893787
Number of SD away 0.7225117401718474

Correct answer  5
Predicted  5
Average  5.056203034403509
Deviation  0.06594687297725182
Number of SD away 0.8522471478351437

Correct answer  4
Predicted  9
Average  8.99999262107849
Deviation  1.0081647781840884e-05
Number of SD away 495949.94085039385

Correct answer  4
Predicted  4
Average  4.000147979119466
Deviation  0.00020880808782534636
Number of SD away 0.7086848072158908

Correct answer  4
Predicted  4
Average  4.015007378927359
Deviation  0.018155330966485368
Number of SD away 0.82661

Correct answer  1
Predicted  1
Average  1.000005671391313
Deviation  6.618750944011826e-06
Number of SD away 0.8568673094057847

Correct answer  4
Predicted  4
Average  4.002335947404265
Deviation  0.002881253105033231
Number of SD away 0.8107400908945316

Correct answer  2
Predicted  2
Average  2.000000118991109
Deviation  1.254178452613538e-07
Number of SD away 0.9487574014535951

Correct answer  6
Predicted  8
Average  8.219432930437627
Deviation  0.3483751961820874
Number of SD away 6.370812143805962

Correct answer  3
Predicted  3
Average  3.0417354764040283
Deviation  0.04927493688424495
Number of SD away 0.8469919809754785

Correct answer  4
Predicted  4
Average  4.455851285262483
Deviation  0.4622325181581095
Number of SD away 0.9861947555722512

Correct answer  7
Predicted  8
Average  8.0
Deviation  0.0
Number of SD away inf

Correct answer  8
Predicted  8
Average  8.000054008442522
Deviation  7.637080540762772e-05
Number of SD away 0.7071870230191005

Correct answer  7
Predic

/home/ug-ml/felix-ML/env/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


...

Correct answer  8
Predicted  8
Average  7.999770420428869
Deviation  0.00034747610220599805
Number of SD away -0.6607060735203422

Correct answer  5
Predicted  5
Average  5.0092335735384435
Deviation  0.01122929334588711
Number of SD away 0.8222755657038188

Correct answer  0
Predicted  0
Average  8.580533166822492e-07
Deviation  1.1635837607069662e-06
Number of SD away 0.7374229046999728

Correct answer  4
Predicted  4
Average  3.991862594896184
Deviation  0.011929191072781358
Number of SD away -0.6821422386621732

Correct answer  5
Predicted  5
Average  5.000002411080755
Deviation  3.021484573192277e-06
Number of SD away 0.7979788399375465

Correct answer  9
Predicted  9
Average  8.99631527682647
Deviation  0.005141657355600989
Number of SD away -0.7166411370286758

Correct answer  0
Predicted  0
Average  4.546929136075213e-05
Deviation  6.201899083451895e-05
Number of SD away 0.7331511001537374

Correct answer  7
Predicted  7
Average  7.004956641739525
Deviation  0.00697358417770029

Correct answer  1
Predicted  1
Average  1.0000006894010947
Deviation  9.833168928589808e-07
Number of SD away 0.7010975807109608

Correct answer  6
Predicted  6
Average  6.000587559596863
Deviation  0.0007861840900045778
Number of SD away 0.7473562545128407

Correct answer  7
Predicted  7
Average  7.0509044920973
Deviation  0.1025019493304275
Number of SD away 0.49661974654943425

Correct answer  1
Predicted  1
Average  1.0000000085423093
Deviation  1.0513426597536375e-08
Number of SD away 0.8125142859095915

Correct answer  4
Predicted  4
Average  4.078999071371072
Deviation  0.10109710214606854
Number of SD away 0.7814177626667445

Correct answer  3
Predicted  2
Average  1.999549087309902
Deviation  0.0010568801322553153
Number of SD away -946.6077392856266

Correct answer  3
Predicted  3
Average  3.0002687938522867
Deviation  0.0003647930065400895
Number of SD away 0.736839378682527

Correct answer  9
Predicted  7
Average  7.0190134372689865
Deviation  0.025022876752708468
Number of

Correct answer  2
Predicted  2
Average  1.9997878523313404
Deviation  0.0005673006767712506
Number of SD away -0.3739598370780133

Correct answer  4
Predicted  4
Average  3.9187759117582397
Deviation  0.16351892716154107
Number of SD away -0.49672591211119355

Correct answer  9
Predicted  8
Average  8.011081874464644
Deviation  0.10440365937670933
Number of SD away -9.472063828406068

Correct answer  0
Predicted  0
Average  0.00016044091017017748
Deviation  0.00022362858855030845
Number of SD away 0.7174436471215486

Correct answer  3
Predicted  3
Average  3.0191322337609114
Deviation  0.022981133214114022
Number of SD away 0.8325191618123187

Correct answer  8
Predicted  8
Average  7.99137743951301
Deviation  0.012228481012515485
Number of SD away -0.7051211412247463

Correct answer  8
Predicted  8
Average  8.000005146496841
Deviation  7.030564124514441e-06
Number of SD away 0.7320176233355463

Correct answer  0
Predicted  0
Average  1.591457210358824e-05
Deviation  2.1858076256242596

Deviation  0.0006621868602662204
Number of SD away -0.8053303702466227

Correct answer  7
Predicted  7
Average  7.000052585688435
Deviation  7.563486312714648e-05
Number of SD away 0.6952572697382096

Correct answer  1
Predicted  1
Average  1.0000683318179684
Deviation  9.677519859034403e-05
Number of SD away 0.7060881193088064

Correct answer  2
Predicted  2
Average  1.9943463635774812
Deviation  0.008930204521781396
Number of SD away -0.6330914828130978

Correct answer  8
Predicted  8
Average  7.999891185177379
Deviation  0.0006074472585808887
Number of SD away -0.1791346015374905

Correct answer  3
Predicted  5
Average  4.9899985458273886
Deviation  0.014033662578543061
Number of SD away 141.80179512581563

Correct answer  5
Predicted  5
Average  5.00012470538612
Deviation  0.0001749385900510895
Number of SD away 0.7128523562658802

Correct answer  5
Predicted  5
Average  5.000008655705215
Deviation  1.2265900729206067e-05
Number of SD away 0.7056722050741545

Correct answer  2
Pred

Correct answer  8
Predicted  8
Average  8.00034498930642
Deviation  0.0008829458846998463
Number of SD away 0.390725312160321

Correct answer  5
Predicted  5
Average  5.000585871302682
Deviation  0.0008260994198288757
Number of SD away 0.7092019297180711

Correct answer  3
Predicted  3
Average  3.000019310651971
Deviation  2.380033611053953e-05
Number of SD away 0.8113604732772628

Correct answer  2
Predicted  7
Average  6.99992491837805
Deviation  0.00040561758038084065
Number of SD away 12326.696771090501

Correct answer  5
Predicted  5
Average  4.972762131722959
Deviation  0.0382102477288868
Number of SD away -0.7128419703086396

Correct answer  2
Predicted  1
Average  1.253999673956514
Deviation  0.2616187843770203
Number of SD away -2.851478451059618

Correct answer  5
Predicted  7
Average  4.944844967452809
Deviation  1.3490052665382006
Number of SD away -0.04088570587180056

Correct answer  5
Predicted  5
Average  5.021237248121797
Deviation  0.029784993165166435
Number of SD aw